#Neural Network Embedding Recommendation System
1. Load in data and clean
2. Prepare data for supervised machine learning task
3. Build the entity embedding neural network
4. Train the neural network on prediction task
5. Extract embeddings and find most similar books and wikilinks
6. Visualize the embeddings using dimension reduction techniques

## 1. Read Data & Clean
: 모든 책의 데이터는 json으로 저장되어 있음. 해당 데이터는 위키피디아의 모든 책에 대한 기사가 포함되어 있음

1-1. 데이터 불러오기

In [39]:
from IPython.core.interactiveshell import InteractiveShell

# Set shell to show all lines of output
# jupyter notebook에서 모든 ourput 나타내기
InteractiveShell.ast_node_interactivity = 'all'

In [40]:
# import os
# print(os.listdir("./input"))

In [41]:
# utils import get_file == url -> file download
import tensorflow as tf
# from keras.utils import get_file

x = tf.keras.utils.get_file('found_books_filtered.ndjson', 'https://raw.githubusercontent.com/WillKoehrsen/wikipedia-data-science/master/data/found_books_filtered.ndjson')

import json

books = []

with open(x, 'r') as fin: # 'r' 읽기용으로 파일 열기
    # Append each line to the books
    books = [json.loads(l) for l in fin]

# Remove non-book articles
books_with_wikipedia = [book for book in books if 'Wikipedia:' in book[0]]
books = [book for book in books if 'Wikipedia:' not in book[0]]
print(f'Found {len(books)} books.')


Found 37020 books.


1-2. 데이터 전처리
: 일부 기사에는 책에대한 기사가 아닌 것들을 잡아낸다.

In [42]:
# book list 내부의 book_with_wikipedia 일부 가져오기
[book[0] for book in books_with_wikipedia][:3]



['Wikipedia:Wikipedia Signpost/2014-06-25/Recent research',
 'Wikipedia:New pages patrol/Unpatrolled articles/December 2010',
 'Wikipedia:Templates for discussion/Log/2012 September 23']

In [43]:
# 제목, 'infoboxs book'의 정보, 위키피디아 링크, 외부링크, 최종수정날짜, 기사의 문자 수
# title, information from 'infobos book' template, wikipedia links, externel links, the date of last edit, the number of characters in article
n = 21
books[n][0], books[n][1], books[n][2][:5], books[n][:5], books[n][:5], books[n][4], books[n][5]

('Limonov (novel)',
 {'author': 'Emmanuel Carrère',
  'country': 'France',
  'english_pub_date': '2014',
  'isbn': '978-2-8180-1405-9',
  'language': 'French',
  'name': 'Limonov',
  'pages': '488',
  'pub_date': '2011',
  'publisher': 'P.O.L.',
  'translator': 'John Lambert'},
 ['Emmanuel Carrère',
  'biographical novel',
  'Emmanuel Carrère',
  'Eduard Limonov',
  'Prix de la langue française'],
 ['Limonov (novel)',
  {'author': 'Emmanuel Carrère',
   'country': 'France',
   'english_pub_date': '2014',
   'isbn': '978-2-8180-1405-9',
   'language': 'French',
   'name': 'Limonov',
   'pages': '488',
   'pub_date': '2011',
   'publisher': 'P.O.L.',
   'translator': 'John Lambert'},
  ['Emmanuel Carrère',
   'biographical novel',
   'Emmanuel Carrère',
   'Eduard Limonov',
   'Prix de la langue française',
   'Prix Renaudot',
   '2011 in literature',
   'Contemporary French literature',
   'Category:2011 French novels',
   'Category:French biographies',
   'Category:Biographical novels'

In [44]:
# 책정보 정수로 변경하기 # index 
book_index = {book[0] : idx for idx, book in enumerate(books)} #enumerate : 인덱스, 원소로 이루어진 tuple로 만들어줌
index_book = {idx : book for book, idx in book_index.items()} # items() : key와 대응값 가져오기 # book_index의 대응값 'title' 가져오기

book_index['Dreaming Spies']
index_book[98]
index_book[100]


98

'Dreaming Spies'

'A Million Ways to Die in the West (novel)'

In [45]:
# Exploring Wikilinks
# chain method = 자기자신을 반환하면서 다른 함수를 지속적으로 호출할 수 있는 방법
from itertools import chain
wikilinks = list(chain(*[book[2] for book in books]))
print(f"There are {len(set(wikilinks))} unique wikilinks.") # set() 중복제거


There are 311276 unique wikilinks.


In [46]:
# 다른책에는 얼마나 많은 wiki link가 있나?
wikilinks_other_book = [link for link in wikilinks if link in book_index.keys()] #link에 key, 대응값이 있다면 wikilinks에서 link list로 뽑아 만든다
print(f"There are {len(set(wikilinks_other_book))} unique wikilinks to other books") # 중복치 제거 길이값

There are 17032 unique wikilinks to other books


In [47]:
# 가장 많이 연결된 기사 찾기
# items 항목 수가 카운트된 dictionary를 반환하는 함수를 만든다.
# collections module : count(개수세기), OrderedDict

from collections import Counter, OrderedDict

def count_items(l):
  # Return ordered dictionary of counts of objects in `l`
  # create a count object
  counts = Counter(l)

  # sort by highest count first and place in orderd dictionary
  # sort(key = (key인자에 함수를 넘겨주면 우선순위가 정해진다))
  counts = sorted(counts.items(), key = lambda x: x[1], reverse = True)  # x[1] 우선순위 숫자로 변경, reverse = 높은 숫자부터
  counts = OrderedDict(counts) # 데이터 순서 설정(key, val)

  return counts


In [48]:
# Find set of wikilinks from each book and convert to a flattend last
# 각각 책에서 wikilinks 설정을 찾고 1차원으로 변경하기

# list(chain(*(set ~~))) = ????

unique_wikilinks = list(chain(*[list(set(book[2])) for book in books])) # books의 중복치를 제거한 wikilinks 값

wikilink_counts = count_items(unique_wikilinks) # 가장 많이 사용된 wikilinks의 unique_counts 상위 10개 불러오기
list(wikilink_counts.items())[:10]

[('Hardcover', 7489),
 ('Paperback', 7311),
 ('Wikipedia:WikiProject Books', 6043),
 ('Wikipedia:WikiProject Novels', 6015),
 ('English language', 4185),
 ('United States', 3060),
 ('Science fiction', 3030),
 ('The New York Times', 2727),
 ('science fiction', 2502),
 ('novel', 1979)]

In [49]:
# 대문자 -> 소문자로 변경하기
wikilinks = [link.lower() for link in unique_wikilinks] # lower() 대문자 -> 소문자 : 동일링크 : paperback, Paperback, PAPERBACK 등 링크 통합
print(f"There are {len(set(wikilinks))} unique wikilinks.")

wikilink_counts = count_items(wikilinks)
list(wikilink_counts.items())[:10]

There are 297624 unique wikilinks.


[('paperback', 8740),
 ('hardcover', 8648),
 ('wikipedia:wikiproject books', 6043),
 ('wikipedia:wikiproject novels', 6016),
 ('science fiction', 5665),
 ('english language', 4248),
 ('united states', 3063),
 ('novel', 2983),
 ('the new york times', 2742),
 ('fantasy', 2003)]

In [50]:
# 가장 유명한 wikilink 제거하기 : 
# 이유 
# 1) 가장 인기있는 위키피디아 링크는 유익한 정보를 가져다주지 않는다.
# 2) 2개의 wikipedia 링크 사용하지 않음(콘텐츠 기반 X)
# 3) 추천에 큰 영향을 미칠 수 있으므로, 가장 인기있는 위키피디아 링크가 제거된 데이터를 사용하는 게 좋다.

to_remove = ['hardcover', 'paperback', 'hardback', 'e-book', 'wikipedia:wikiproject books', 'wikipedia:wikiproject novels'] # e-book, hardback은 왜 제거하는지 모르겠음

for t in to_remove:
    wikilinks.remove(t)
    _ = wikilink_counts.pop(t) # ????? #pop(t) t가 들어간 to_move의 카테고리들을 제거해라

In [51]:
# 4번 이상 나온 wikilinks를 사용한다.

links = [t[0] for t in wikilink_counts.items() if t[1] >= 4] # ?????
print(len(links))

41758


In [52]:
# wikipedia에서 언급된 다른 책
# 각 책에대한 book wikilinks 찾기
unique_wikilinks_books = list(chain(*[list(set(link for link in book[2] if link in book_index.keys())) for book in books])) # * 모든것 : chain(10) -> 10 11 12 13 14 이어붙이기 

# 다른책에서 링크된 책의 숫자
wikilinks_book_counts = count_items(unique_wikilinks_books)
list(wikilinks_book_counts.items())[:10]



[('The Encyclopedia of Science Fiction', 127),
 ('The Discontinuity Guide', 104),
 ('The Encyclopedia of Fantasy', 63),
 ('Dracula', 55),
 ('Encyclopædia Britannica', 51),
 ('Nineteen Eighty-Four', 51),
 ('Don Quixote', 49),
 ('The Wonderful Wizard of Oz', 49),
 ("Alice's Adventures in Wonderland", 47),
 ('Jane Eyre', 39)]

In [56]:
# 잠재적인 추가제거 작업
# 데이터 전처리를 추가적으로 진행하고 싶다면 수행
for book in books:
    if 'The New York Times' in book[2] and 'New York Times' in book[2]:
        print(book[0], book[2])
        break




The Big Picture: Who Killed Hollywood? and Other Essays ['Wikipedia:WikiProject Novels', 'Wikipedia:WikiProject Books', 'William Goldman', 'United States', 'English language', 'William Goldman', 'Michael Sragow', 'Good Will Hunting', 'Robin Williams', 'Matt Damon', 'The New York Times', 'The New York Times Company', 'New York Times', 'Category:Cinema of the United States', 'Category:Film production', 'Category:2000 books', 'Category:Books about films', 'Category:Books by William Goldman', 'Category:Show business memoirs']


In [59]:
wikilink_counts.get('the new york times')

wikilink_counts.get('new york times')

2742

996

In [60]:
# Wikilinks to Index
# book 데이터를 정수로 바꾸어주었듯이, Wikilinks도 정수로 바꿔준다
link_index = {link: idx for idx, link in enumerate(links)}
index_link = {idx: link for link, idx in link_index.items()}

link_index['the economist']
index_link[300]
print(f'There are {len(link_index)} wikilinks that will be used.')


300

'the economist'

There are 41758 wikilinks that will be used.
